# Limpieza

## Parámetros

In [1]:
BASE_DIR = '/Users/efraflores/Desktop/EF/Contests/Ecobici/data'

## Importar

In [2]:
from ecobici.models import EcoBici

eb = EcoBici(BASE_DIR)
print(eb)

44 archivos en:
/Users/efraflores/Desktop/EF/Contests/Ecobici/data


In [3]:
df = eb.read_raw_files(from_year=2018, to_year=2021, index=['Ciclo_Estacion_Retiro','fecha'], columns='fecha_hour_range', values='n', aggfunc='count', fill_value=0)
print(df.shape)
df.sample()

Archivo: EcoBici_from_2018_to_2021.csv fue exportado exitosamente en:
/Users/efraflores/Desktop/EF/Contests/Ecobici/data/transformed
(629497, 9)


fecha_hour_range,Ciclo_Estacion_Retiro,fecha,00 a 09,10 a 12,13 a 14,15 a 17,18 a 20,21 a 1000,>= 21
167490,128.0,2020-12-31,3,1,3,6,0,0.0,0.0


## Transformar

In [4]:
X, y = eb.ecobici_shifted(df, id_cols=['Ciclo_Estacion_Retiro'], shifts=range(41))
print(X.shape)
X.sample()

Archivo: EcoBici_shifted.csv fue exportado exitosamente en:
/Users/efraflores/Desktop/EF/Contests/Ecobici/data/transformed
(616938, 280)


,,00 a 09_01,00 a 09_02,00 a 09_03,00 a 09_04,00 a 09_05,00 a 09_06,00 a 09_07,00 a 09_08,00 a 09_09,00 a 09_10,...,>= 21_31,>= 21_32,>= 21_33,>= 21_34,>= 21_35,>= 21_36,>= 21_37,>= 21_38,>= 21_39,>= 21_40
Ciclo_Estacion_Retiro,fecha,,,,,,,,,,,,,,,,,,,,,
417.0,2019-03-24,3.0,6.0,6.0,6.0,8.0,1.0,0.0,1.0,6.0,8.0,...,0.0,5.0,2.0,2.0,0.0,0.0,0.0,2.0,0.0,3.0


## Modelo

### Entrenamiento

In [5]:
model, scores, coef = eb.train_model(X, y)

Test score:	89.03%
Train score:	88.92%


### Variables más importantes

In [6]:
n_vars = 10

coef.head(n_vars//2).append(coef.tail(n_vars//2)).style.format("{:.2f}")

,Peso
Variable,
18 a 20_01,3.80
00 a 09_01,3.73
00 a 09_07,3.26
15 a 17_01,2.66
15 a 17_07,2.13
00 a 09_16,-0.59
00 a 09_37,-0.62
15 a 17_39,-0.64
00 a 09_29,-0.64


### Real vs estimado

In [7]:
eb.plot_real_vs_est(X, y, model, id_col='Ciclo_Estacion_Retiro')

## Preservación del Modelo

In [8]:
eb.save_model({'pronostico':model, 'pronostico_cols':X.columns}, 'ecobici_modelo_pronostico')

El modelo ecobici_modelo_pronostico.xz fue guardado existosamente en:
/Users/efraflores/Desktop/EF/Contests/Ecobici/data/models
